# Spread Trading Strategy
Implementation of a spread trading strategy using the spread between the prices of two futures. The strategy is based on the assumption that the spread between the two stocks will revert to the mean. The strategy is implemented using the following steps:
- Calculate the spread between the two futures: $p_{\Delta} = R \times p_a - p_b$
- Calculate the mean and standard deviation of the spread: $\mu_{\Delta}$ and $\sigma_{\Delta}$
- Notional Exposure per synthetic contract: $= p_{\Delta} \times ($ Futures multiplier $\times$ FX rate $)$ 

**Formulas to Utilize:**
- $\sigma_{portfolio} = \sqrt{\sum_{i=1}^{n} w_i^2 \sigma_i^2 + \sum_{i=1}^{n}\sum_{j=1}^{n} w_i w_j \sigma_i \sigma_j \rho_{ij}}$
- $\rho_{ab} = \frac{Cov(a,b)}{\sigma_a \sigma_b}$
- $\sigma^{\Delta}_{p} = \sqrt{R^{2}\sigma_a^2 + \sigma_b^2 - 2 \sigma_a \sigma_b \rho_{ab}}$


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# Spread of SP500 and US10Y
sp500 = pd.read_csv('sp500.csv', index_col=0, parse_dates=True)
us10y = pd.read_csv('us10.csv', index_col=0, parse_dates=True)
sp500.tail(), us10y.tail()

(            adjusted  underlying
 index                           
 2022-09-26   3667.25     3667.25
 2022-09-27   3665.25     3665.25
 2022-09-28   3729.75     3729.75
 2022-09-29   3658.75     3658.75
 2022-09-30   3641.50     3641.50,
               adjusted  underlying
 index                             
 2022-09-26  111.046875  111.046875
 2022-09-27  111.031250  111.031250
 2022-09-28  112.656250  112.656250
 2022-09-29  112.140625  112.140625
 2022-09-30  112.484375  112.484375)

In [28]:
"""
The SP500 will be our long and we will assume that the US10Y is our short
Our R-value or Ratio will be 2 to make the math easier
Our price of the spread will be the SP500 - 2 * US10Y
"""
ratio = 20
p_delta = sp500['adjusted'] - ratio * us10y['underlying']
p_delta.tail()

index
2022-09-26    1446.3125
2022-09-27    1444.6250
2022-09-28    1476.6250
2022-09-29    1415.9375
2022-09-30    1391.8125
dtype: float64

In [29]:
sp500_array = np.array(sp500['adjusted'])
us10_array = np.array(us10y['adjusted'])
length = min(len(sp500_array), len(us10_array))
sp500_array = sp500_array[-length:]
us10_array = us10_array[-length:]
rho = np.corrcoef(sp500_array, us10_array)[0,1]
round(rho, 4)

0.7659

In [32]:
var_sp500 = np.var(sp500_array)
var_us10 = np.var(us10_array)
std_sp500 = np.std(sp500_array)
std_us10 = np.std(us10_array)
np.sqrt(var_sp500+ ratio**2 * var_us10 - 2 * ratio * rho * np.sqrt(var_sp500 * var_us10))

666.560976620281